In [1]:
import tkinter as tk
from tkinter import messagebox
import subprocess
import os
import sys

import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageTk

In [2]:
# Prevent TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
# Load models

# Load the trained model
model_path_cnn = os.path.abspath("../model/cnn.keras")
cnn_model = tf.keras.models.load_model(model_path_cnn)

model_path_cnn1 = os.path.abspath("../model/cnn_1.keras")
cnn_aug_model = tf.keras.models.load_model(model_path_cnn1)

model_path_vgg16 = os.path.abspath("../model/vgg16.keras")
vgg16_model = tf.keras.models.load_model(model_path_vgg16)

model_path_vgg16_1 = os.path.abspath("../model/vgg16_finetuned.keras")
vgg16_1_model = tf.keras.models.load_model(model_path_vgg16_1)

In [4]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import tensorflow as tf


# Labels used during training
labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ["space", "del", "nothing"]
IMG_SIZE = 64

# Webcam setup
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# GUI Setup
root = tk.Tk()
root.title("ASL Prediction GUI")
root.geometry("820x540")  # Adjust window height

video_frame = tk.Label(root)
video_frame.pack(pady=10)

capture_btn = tk.Button(root, text="📷 Capture", font=("Helvetica", 14), command=lambda: capture_and_predict())
capture_btn.pack(pady=10)

overlay_roi = None
overlay_text = ""

def draw_bounding_box(frame, box_size=250):
    h, w, _ = frame.shape
    x1 = w // 2 - box_size // 2
    y1 = h // 2 - box_size // 2
    x2 = x1 + box_size
    y2 = y1 + box_size
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return frame, (x1, y1, x2, y2)

def update_webcam():
    global overlay_roi, overlay_text
    ret, frame = cap.read()
    if not ret:
        return
    frame = cv2.flip(frame, 1)
    frame, _ = draw_bounding_box(frame)

    # Overlay ROI preview and prediction text
    if overlay_roi is not None:
        roi_resized = cv2.resize(overlay_roi, (100, 100))
        frame[10:110, 10:110] = roi_resized  # Top-left

        if overlay_text:
            y_offset = 120
            for line in overlay_text.strip().split("  "):
                cv2.putText(frame, line.strip(), (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                            0.45, (0, 0, 0), 1, cv2.LINE_AA)
                y_offset += 18

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = ImageTk.PhotoImage(Image.fromarray(img))
    video_frame.configure(image=img)
    video_frame.image = img
    root.after(10, update_webcam)

def predict_from_models(frame, box_coords):
    preds = {}

    # CNN and AugCNN (grayscale, 64x64, 1)
    x1, y1, x2, y2 = box_coords
    roi_gray = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    roi_gray_resized = cv2.resize(roi_gray, (64, 64))
    roi_gray_normalized = roi_gray_resized.astype("float32") / 255.0
    roi_gray_input = np.expand_dims(roi_gray_normalized, axis=(0, -1))  # (1, 64, 64, 1)

    for model_name, model in [("CNN", cnn_model), ("AugCNN", cnn_aug_model)]:
        prob = model.predict(roi_gray_input, verbose=0)[0]
        pred_idx = np.argmax(prob)
        preds[model_name] = (labels[pred_idx], prob[pred_idx])

    # VGG models (RGB, 96x96, 3)
    roi_rgb = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2RGB)
    roi_rgb_resized = cv2.resize(roi_rgb, (96, 96))
    roi_rgb_normalized = roi_rgb_resized.astype("float32") / 255.0
    roi_rgb_input = np.expand_dims(roi_rgb_normalized, axis=0)  # (1, 96, 96, 3)

    for model_name, model in [("VGG16", vgg16_model), ("VGG16_FT", vgg16_1_model)]:
        prob = model.predict(roi_rgb_input, verbose=0)[0]
        pred_idx = np.argmax(prob)
        preds[model_name] = (labels[pred_idx], prob[pred_idx])

    return preds

def capture_and_predict():
    global overlay_roi, overlay_text
    ret, frame = cap.read()
    if not ret:
        messagebox.showerror("Error", "Failed to read from webcam.")
        return
    frame = cv2.flip(frame, 1)
    frame_boxed, box_coords = draw_bounding_box(frame)

    # Save ROI
    x1, y1, x2, y2 = box_coords
    overlay_roi = frame_boxed[y1:y2, x1:x2]

    # Predict
    predictions = predict_from_models(frame, box_coords)
    overlay_text = ""
    for model, (label, conf) in predictions.items():
        overlay_text += f"{model}: {label} ({conf:.2f})  "

def on_close():
    cap.release()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_close)

# Start
update_webcam()
root.mainloop()


2025-07-29 12:20:05.207856: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.
